<a href="https://colab.research.google.com/github/clashleyuncc/Intro-To-ML/blob/main/HW6P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#P3a

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
import time
start_time = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

def softmax(x):
  return torch.exp(x) / torch.exp(x).sum()

def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)

        t_u_train_device = t_u_train.to(device)
        t_c_train_device = t_c_train.to(device)

        t_p_train = model(t_u_train_device)
        loss_train = loss_fn(t_p_train, t_c_train_device)

        t_u_val_device = t_u_val.to(device)
        t_c_val_device = t_c_val.to(device)

        t_p_val = model(t_u_val_device)
        loss_val = loss_fn(t_p_val, t_c_val_device)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        train_losses.append(loss_train)
        val_losses.append(loss_val)

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")
            print(f"Training Time: {time.time()-start_time} seconds")


data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

#img, label = cifar10[99]

from torchvision import transforms

to_tensor = transforms.ToTensor()
#img_t = to_tensor(img)

tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
                                  transform=transforms.ToTensor())

#imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)

#transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))

transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.4915, 0.4823, 0.4468),
                            (0.2470, 0.2435, 0.2616))
    ]))

img_t, _ = transformed_cifar10[99]
#plt.imshow(img_t.permute(1,2,0))
#plt.show()

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)

seq_model = nn.Sequential(OrderedDict([
            ('hidden_linear1', nn.Linear(3072, 512)), #input size?
            ('hidden_activation1', nn.Tanh()),
            ('output_linear', nn.Linear(512,10)),
            ('softmax', nn.LogSoftmax(dim=1))
]))

seq_model.to(device)
optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)

loss_fn = nn.NLLLoss()
n_epochs = 100

for epoch in range(n_epochs):
  for imgs, labels in train_loader:
    imgs, labels = imgs.to(device), labels.to(device)
    batch_size = imgs.shape[0]
    outputs = seq_model(imgs.view(batch_size, -1))
    loss = loss_fn((outputs, labels))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print("Epoch: %d, Loss:  %f" % (epoch, float(loss)))




Using device: cpu
Files already downloaded and verified
Files already downloaded and verified


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [3]:
import torch
print(torch.__version__)


#import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
import time
start_time = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

from torchvision import transforms

#tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
#                                  transform=transforms.ToTensor())

#transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))

transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.4915, 0.4823, 0.4468),
                            (0.2470, 0.2435, 0.2616))
    ]))

#img_t, _ = transformed_cifar10[99]

train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64,
                                           shuffle=True)

seq_model = nn.Sequential(OrderedDict([
            ('hidden_linear1', nn.Linear(3072, 512)), #input size?
            ('hidden_activation1', nn.Tanh()),
            ('output_linear', nn.Linear(512,10)),
            ('softmax', nn.LogSoftmax(dim=1))
]))

#seq_model.to(device)
optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)

loss_fn = nn.NLLLoss()
n_epochs = 100

for epoch in range(1, n_epochs+1):
  for imgs, labels in train_loader:
    imgs, labels = imgs.to(device), labels.to(device)
    batch_size = imgs.shape[0]
    outputs = seq_model(imgs.view(batch_size, -1))
    loss = loss_fn(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print("Epoch: %d, Loss:  %f" % (epoch, float(loss)))

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)
correct = 0
total = 0

with torch.no_grad():
  for imgs, labels in val_loader:
    imgs, labels = imgs.to(device), labels.to(device)
    batch_size = imgs.shape[0]
    outputs = seq_model(imgs.view(batch_size, -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += int((predicted == labels).sum())
print("Accuracy: %f", correct / total)
print(f"Training Time: {time.time()-start_time} seconds")

2.5.1+cu121
Using device: cpu
Files already downloaded and verified
Files already downloaded and verified
Epoch: 1, Loss:  1.814229
Epoch: 2, Loss:  1.937203
Epoch: 3, Loss:  1.927518
Epoch: 4, Loss:  1.502651
Epoch: 5, Loss:  1.980931
Epoch: 6, Loss:  1.367940
Epoch: 7, Loss:  1.189667
Epoch: 8, Loss:  2.052720
Epoch: 9, Loss:  1.649181
Epoch: 10, Loss:  1.441229
Epoch: 11, Loss:  1.672809
Epoch: 12, Loss:  1.615952
Epoch: 13, Loss:  1.418135
Epoch: 14, Loss:  1.629136
Epoch: 15, Loss:  1.545108
Epoch: 16, Loss:  1.939107
Epoch: 17, Loss:  2.052490
Epoch: 18, Loss:  1.301529
Epoch: 19, Loss:  1.178459
Epoch: 20, Loss:  1.372431
Epoch: 21, Loss:  1.259214
Epoch: 22, Loss:  1.330530
Epoch: 23, Loss:  1.596835
Epoch: 24, Loss:  1.540686
Epoch: 25, Loss:  1.760687
Epoch: 26, Loss:  1.518540
Epoch: 27, Loss:  1.272394
Epoch: 28, Loss:  1.850298
Epoch: 29, Loss:  1.221365
Epoch: 30, Loss:  1.343146
Epoch: 31, Loss:  1.380065
Epoch: 32, Loss:  1.009019
Epoch: 33, Loss:  1.713693
Epoch: 34, L

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
import time
start_time = time.time()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

from torchvision import transforms

#tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False,
#                                  transform=transforms.ToTensor())

#transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))

transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.4915, 0.4823, 0.4468),
                            (0.2470, 0.2435, 0.2616))
    ]))

#img_t, _ = transformed_cifar10[99]

train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size=64,
                                           shuffle=True)

seq_model = nn.Sequential(OrderedDict([
            ('hidden_linear1', nn.Linear(3072, 512)),
            ('hidden_activation1', nn.Tanh()),
            ('hidden_linear1', nn.Linear(3072, 512)),
            ('hidden_activation1', nn.Tanh()),
            ('hidden_linear1', nn.Linear(3072, 512)),
            ('hidden_activation1', nn.Tanh()),
            ('output_linear', nn.Linear(512,10)),
            ('softmax', nn.LogSoftmax(dim=1))
]))

#seq_model.to(device)
optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)

loss_fn = nn.NLLLoss()
n_epochs = 100

for epoch in range(1, n_epochs+1):
  for imgs, labels in train_loader:
    imgs, labels = imgs.to(device), labels.to(device)
    batch_size = imgs.shape[0]
    outputs = seq_model(imgs.view(batch_size, -1))
    loss = loss_fn(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print("Epoch: %d, Loss:  %f" % (epoch, float(loss)))

val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)
correct = 0
total = 0

with torch.no_grad():
  for imgs, labels in val_loader:
    imgs, labels = imgs.to(device), labels.to(device)
    batch_size = imgs.shape[0]
    outputs = seq_model(imgs.view(batch_size, -1))
    _, predicted = torch.max(outputs, dim=1)
    total += labels.shape[0]
    correct += int((predicted == labels).sum())
print("Accuracy: %f", correct / total)
print(f"Training Time: {time.time()-start_time} seconds")